In [1]:
import pandas as pd
import numpy as np
import tabula
import math
import os

In [2]:
#este bloque busca los BOD en PDF en la carpeta de documentos (docs_dir), los compara con los documentos que ya se procesaron en su día guardados como .pkl (pkld_list) y genera una lista
#de nombres de archivos a procesar (proc_docs)

docs_dir = 'D:/jaume/Datasets/BOD2/'
pkld_dir = 'D:/jaume/Jupyter Notebooks/Vacantes Scraper/ScrapedData2/'
proc_doc = 'BOD_20210325_58.pdf'

docs_list = []    #lista de documentos en la carpeta de entrada
pkld_list = []    #lista de documentos ya procesados y transformados a pkl
proc_docs = [proc_doc]    #lista de documentos aún por procesar

# # iterate over files in that directory
# for filename in os.scandir(docs_dir):
#     if filename.is_file():
#         docs_list.append(filename.name.split('_')[1])
        
# # iterate over files in that directory
# for filename in os.scandir(pkld_dir):
#     if filename.is_file():
#         pkld_list.append(filename.name.split('_')[0])

# #seriales de documentos a procesar
# pending_list = [doc for doc in docs_list if doc not in pkld_list]

# # list of files to porcess
# for filename in os.scandir(docs_dir):
#     if filename.name.split('_')[1] in pending_list:
#         proc_docs.append(filename)

In [21]:
for doc in proc_docs:
    
    print('Processing ' + doc + '...')
    file_pdf = docs_dir + doc
    read_pdf = tabula.read_pdf(file_pdf, pages = 'all', silent = True)
    
    sel_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el número de filas (2)
    oth_tables = []                         #creará una lista (sel_tables) con los índices a mantener basado en el número de columnas de la tabla (14-16) y el más de 2 filas (funcionan diferente)
    for i in range(len(read_pdf)):
        if np.logical_and(read_pdf[i].shape[0] == 2, np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):         #seleccionamos sólo los df que contienen de 14 a 16 columnas y 2 filas
            sel_tables.append(i)

        elif np.logical_and(read_pdf[i].shape[0] > 2, np.logical_and(read_pdf[i].shape[1] >= 14, read_pdf[i].shape[1] <= 16)):         #seleccionamos sólo los df que contienen de 14 a 16 columnas y 2 filas
            oth_tables.append(i)    

        else:
            continue

    df_list = [read_pdf[index] for index in sel_tables]    #conservará los elementos de read_tables seleccionados en sel_tables
    oth_list = [read_pdf[index] for index in oth_tables]    #conservará los elementos de read_tables seleccionados en sel_tables

    df_clean = []
    oth_clean = []
    errors = []

    #código para los df de shape (2, 14-16) almacenados en df_list

    for i in range(len(df_list)):
        df = df_list[i]                                   #iteramos los df seleccionados en df_list
        col_list =[]                                      #instanciamos la lista de columnas vacía que se poblará con los 'split' de cada columna del df de esta iteración
        string = [np.nan] * df.shape[1]
        col = [np.nan] * df.shape[1]
        
        if isinstance(df.iloc[1, 0], str):                    #n_vacs define el número de vacantes que se esperan (num de vacantes separadas por \r en la 1a col del DF)
            n_vacs = len(df.iloc[1, 0].split('\r'))

        elif isinstance(df.iloc[1, 0], float):
            n_vacs = 1

        else:
            print('DF-' + str(i) + 'found a ' + str(type(df.iloc[1, 0])) + ' in iloc [1, 0]')

        for j in range(df.shape[1]):                      #iteramos cada columna (str separado por '\r') del df para convertirlo en una lista de valores de la columna

            string[j] = df.iloc[1, j]

            if type(string[j]) == str:

                col[j] = string[j].split('\r')            #columna resultante de la separación de 'string'

                if len(col[j]) == n_vacs:
                    col_list.append(col[j])               #si la columna tiene exactamente el mismo número de registros que el número de vacantes del df, adjuntamos la columna

                elif math.ceil(len(col[j])/2) == n_vacs:
                    col_list.append(col[j][0::2])         #si la columna tiene el doble (redondeado) de registros que el número de vacantes del df, adjuntamos la columna cada dos espacios

                else:                                     #si no tiene el mismo número, lanzamos la lógica para colocar todos los valores posibles en ese registro y marcamos el error
                    errors.append('DF-' + str(i) + ' Elementos no definidos en todas las vacantes. Ver opciones.')
                    unique = set(col[j])
                    values = list(unique)
                    values.append(np.nan)
                    col_list.append([values] * n_vacs)
            
            elif type(string[j]) == np.float64:
                col[j] = string[j]
                col_list.append(col[j])
            
            else:
                col_list.append([np.nan] * n_vacs)

        col_map = {}
        if len(col_list) == 14:                             #mapeo según resoluciones de 14 columnas
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'empleo': col_list[6],
                       'efun': col_list[7],
                       'cursos': col_list[8],
                       't_max': col_list[9],
                       't_min': col_list[10],
                       'fecha_cob': col_list[11],
                       'csce': col_list[12],
                       'obs': col_list[13]
                      }

        elif np.logical_and(len(col_list) == 15, 'CM' in col_list[4]):                           #mapeo según resoluciones de 15 columnas agregar condición 'y en la columna 4 contiene CM' para deconflictar con vacantes de RESERVA
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'empleo': col_list[6],
                       'efun': col_list[7],
                       'cursos': col_list[8],
                       't_max': col_list[9],
                       't_min': col_list[10],
                       'fecha_cob': col_list[11],
                       'csce': col_list[12],
                       'cod_cm': col_list[13],
                       'obs': col_list[14]
                      }
            
        elif np.logical_and(len(col_list) == 15, 'CM' not in col_list[4]):                           #mapeo según resoluciones de 15 columnas diferentes a CM (pendiente mapeo)
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'empleo': col_list[7],
                       'csce': col_list[13],
                       'obs': col_list[14]
                      }

        elif len(col_list) == 16:                           #mapeo según resoluciones de 16 columnas
            col_map = {'n_vac': col_list[0],
                       'uco': col_list[1],
                       'ciu': col_list[2],
                       'pt': col_list[3],
                       'asig': col_list[4],
                       'ta': col_list[5],
                       'ejercito': col_list[6],
                       'cuerpo_esc': col_list[7],
                       'empleo': col_list[8],
                       'efun': col_list[9],
                       'cursos': col_list[10],
                       't_max': col_list[11],
                       't_min': col_list[12],
                       'nivel': col_list[13],   #comprobar que esto es así siempre.
                       'csce': col_list[14],
                       'obs': col_list[15]
                      }

        else:
            errors.append('DF-' + str(i) + ' has wrong COL MAPPING')


        try:
            df_clean.append(pd.DataFrame(col_map))

        except:
            errors.append('DF-' + str(i) + ' threw an error in DataFrame')



    print(str(len(df_clean)) + ' clean df,s in the list, ready to pack')

    #código para los df de shape (2+, 14-16) almacenados en oth_list

    for oth_df in oth_list:

        oth_df = oth_df.iloc[1::2]

        vac_uco = oth_df.iloc[:, 0].str.split(n = 1, expand = True)
        n_vac = vac_uco.iloc[:, 0]
        uco = vac_uco.iloc[:, 1]
        
        
        
        clas_ta = oth_df['CLAS.'].str.split(n = 1, expand = True)    
        asig = clas_ta.iloc[:, 0]
        ta = clas_ta.iloc[:, 1]
        
        obs = oth_df.iloc[:, -1]

        assert len(n_vac) == len(uco) == len(asig) == len(ta)        #checkea que hay tantos n_vac como ucos, modos de asignación y TAs
        
        pt_ind = 0
        ciu_ind = 0
        cm_ind = 0
        csce_ind = 0
        empleo_ind = 0

        for i in range(oth_df.shape[1]):

                if all([isinstance(e, str) for e in oth_df.iloc[:, i]]):
                    if round(oth_df.iloc[:, i].str.len().mean()) == 8:
                        ciu_ind = i
  
                    elif oth_df.iloc[:, i].str.count('/').sum() == len(oth_df.iloc[:, i]):
                        pt_ind = i
   
                    elif np.logical_and(oth_df.iloc[:, i].str.len().mean() == 4, ~oth_df.iloc[:, i].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').all()):
                        cm_ind = i
          
                    elif oth_df.iloc[:, i].str.len().mean() == 6:
                        csce_ind = i
            
                    elif oth_df.iloc[:, i].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').sum().item() == len(oth_df.iloc[:, i]):
                        empleo_ind = i
                        
                    else: pass
        

        col_map = {'n_vac': n_vac,
               'uco': uco,
               'ciu': oth_df.iloc[:, ciu_ind],
               'pt': oth_df.iloc[:, pt_ind],
               'asig': asig,
               'ta': ta,
               'empleo': oth_df.iloc[:, empleo_ind],
    #           'efun': col_list[7],
    #           'cursos': col_list[8],
    #           't_max': col_list[9],
    #           't_min': col_list[10],
    #           'fecha_cob': col_list[11],
               'csce': oth_df.iloc[:, csce_ind],
               'cod_cm': oth_df.iloc[:, cm_ind],
               'obs': obs
              }


        oth_clean.append(pd.DataFrame(col_map))


        errors.append('DF_OTH-' + str(i) + ' threw an error in DataFrame')

    print(str(len(df_clean)+len(oth_clean)) + ' clean df,s in 2 lists, ready to pack')

    try:
        df_full = pd.concat(df_clean + oth_clean).reset_index(drop = True)      #la solución!!!
    except: pass
    
    pkl_name_parts = file_pdf.split('_')
    pkl_name = pkl_name_parts[1] + '.pkl'

    df_full.to_pickle(pkld_dir + pkl_name)

    print(str(pkl_name_parts[1]) + '_' + str(pkl_name_parts[2]) + ' pickled!')

Processing BOD_20210325_58.pdf...
35 clean df,s in the list, ready to pack
38 clean df,s in 2 lists, ready to pack
20210325_58.pdf pickled!


In [173]:
oth_df = oth_list[0][1::2]
vac_uco = oth_df.iloc[:, 0].str.split(n = 1, expand = True)
n_vac = vac_uco.iloc[:, 0].astype('int')
uco = vac_uco.iloc[:, 1]

In [182]:
oth_df.iloc[:, 5].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').sum().item() == len(oth_df.iloc[:, 5])

True

In [183]:
type(oth_df.iloc[:, 5].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').sum().item())

int

In [193]:
~oth_df.iloc[:, 5].str.contains('SDO|CBO|CBO 1º|CBMY|SGTO|SGTO 1º|BG|STTE|SBMY|ALF|TTE|CAP|CTE|TCOL|COL').all()

False

In [185]:
oth_df.iloc[:, 5]

1     STTE
3     STTE
5     STTE
7     STTE
9     STTE
11    STTE
13    STTE
15    STTE
17    STTE
19    STTE
21    STTE
23    STTE
25    STTE
27    STTE
29    STTE
31    STTE
33    STTE
35    STTE
37    STTE
39    STTE
41    STTE
43    STTE
45    STTE
47    STTE
49    STTE
51    STTE
53    STTE
Name: Unnamed: 1, dtype: object

In [17]:
df_list[-14].iloc[:, 14]


0                                                  NaN
1    960\r960\r960\r960\r960\r960\r960\r960\r305,96...
Name: Unnamed: 1, dtype: object

In [110]:
df_list[-20]

,NUM.\rNúVAmC. .,"DIRECCIÓN GENERAL, ORGANISMO, CENTRO, UNIDAD\r8\rLOCALIDADJueves, 25 de",CODIGO\raIDErNzTIoF.\rUNIDAD,CODIGO\re P2UE0ST2O 1\rTRABAJO,CLAS.,EMPLEO,ESP. FUNDAMENTAL,TITUL. / SEG. ESP. EXIGIBLES,TIEMPOS,FECHA\rCOBERTURA,CSCE\rEUROS/MES,TIPO\rCM,OBSERVACIONES,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,AS,TA,ESP. SCOeMPcLE.M EINITIA.R IA P,gTIT.U L.7 / S2EG6. E4SP. VALORABLES,MAX,MIN,NaN,NaN,NaN,NaN,NaN,NaN
1,03616\r03617\r03618\r03619\r03620\r03621\r0362...,USBA «EL GOLOSO»\rMADRID\rUSBA «GENERAL CAVALC...,54130013\r54130015\r54164711\r54217747\r542300...,5SA50/001\r5SA50/004\r5SA50/002\r5SA50/002\r5S...,CM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM\rCM...,C\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC\rC...,BG\rBG\rBG\rBG\rBG\rBG\rBG\rBG\rBG\rBG\rBG\rBG...,NaN,NaN,10\r10\r10\r10\r10\r10\r10\r10\r10\r10\r10\r10...,NaN,NaN,"207,22\r207,22\r207,22\r207,22\r184,24\r207,22...",GM29\rGM29\rGM29\rGM29\rGM29\rGM29\rGM29\rGM29...,"196,511,960\r196,511,960\r196,511,960\r196,511..."


In [22]:
df_full.to_csv('tst.csv')

In [57]:
df_list[1].iloc[1, 0]

'02512\r02513\r02514\r02515\r02516\r02517\r02518\r02519\r02520\r02521\r02522\r02523\r02524\r02525\r02526\r02527\r02528\r02529\r02530\r02531\r02532\r02533'

In [7]:
errors

['DF-0 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-0 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-0 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-1 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-1 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-2 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-2 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-3 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-3 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-4 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-4 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-5 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-5 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-6 Elementos no definidos en todas las vacantes. Ver opciones.',
 'DF-6 Elementos no definidos en t